In [1]:
import torch
import torch.nn as nn

In [2]:
torch.manual_seed(0)

In [3]:
# create RNN network
# - embedding dimension: 4
# - hidden dimension: 2
# - num layers: 1
rnn = nn.RNN(4, 2, 1)

In [4]:
# test input
# - seq len: 2
# - batch size: 1
# - embedding dimension: 10
sample0 = [[1, 1, 1, 1], [2, 2, 2, 2]]  # PADDING なし
sample1 = [[1, 1, 1, 1], [0, 0, 0, 0]]  # 二番目の要素が PADDING
x_ = torch.tensor([sample0, sample1], dtype=torch.float32)
lengths = torch.tensor([2, 1])  # サンプルの要素数。（注意：降順にする）

# RNN への入力を作成
# dim: (seq length, batch_size, embedding dim)
x = x_.transpose(0, 1)

print(x)

tensor([[[ 1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.]],

        [[ 2.,  2.,  2.,  2.],
         [ 0.,  0.,  0.,  0.]]])


In [5]:
# パディングを考慮しない場合
with torch.no_grad():
    y, hidden = rnn(x)
    print(y)
    print(hidden)


tensor([[[-0.9348,  0.0218],
         [-0.9348,  0.0218]],

        [[-0.9824,  0.5931],
         [-0.7184, -0.2406]]])
tensor([[[-0.9824,  0.5931],
         [-0.7184, -0.2406]]])


In [6]:
# パディングを考慮する場合
# PADDING (zero vector) は無視され RNN を通らない。
# それを確認するには、sample1 の最終の hidden state [-0.9348,  0.0218] が、
# sample1 の一番目の入力 [1, 1, 1, 1] に対する出力 [-0.9348,  0.0218] と
# 一致していることを確かめれば良い
packed_x = nn.utils.rnn.pack_padded_sequence(x, lengths)
with torch.no_grad():
    packed_y, hidden = rnn(packed_x)
    out, _ = nn.utils.rnn.pad_packed_sequence(packed_y, total_length=x.shape[0])
    print(out)
    print(hidden)


tensor([[[-0.9348,  0.0218],
         [-0.9348,  0.0218]],

        [[-0.9824,  0.5931],
         [ 0.0000,  0.0000]]])
tensor([[[-0.9824,  0.5931],
         [-0.9348,  0.0218]]])


In [7]:
# 最終出力を取り出す
# 分類問題では、RNN の最終出力に対して分類を実行する必要がある。
# その場合は次のように lengths に対応する出力をインデックスで指定すればよい
out_idx = [(i, idx.item()-1) for i, idx in enumerate(lengths)]
out[out_idx]

tensor([[-0.9348,  0.0218],
        [-0.9824,  0.5931]])